In [2]:
import numpy as np
import pandas as pd
from pyecharts.charts import Map
import pyecharts.options as opts
from pyecharts.options.global_options import ThemeType
code_name=pd.read_csv("./index.csv",encoding='utf-8-sig')   # 读入文件index.csv
map_name = pd.read_csv("./cancer-deaths-by-type.csv",encoding='utf-8-sig')  # 读入文件cancer-deaths-by-type.csv

In [3]:
# Build dictionary

country_code = code_name['country_code'].unique().tolist()  # country_code为所有国家的代码
country_name = code_name['country_name'].unique().tolist()  # country_name为所有国家的名称
country_name1 = []  # country_name1为将United States of America变为United States之后的所有国家的名称
country_name.index('United States of America')
country_name1 = country_name[:228]
country_name1.append('United States')
country_name1 += country_name[229:]

name_dict = dict(zip(country_name1,country_code))   # name_dict为键为国家名称，值为国家代码的字典
name_dict['Namibia'] = 'NA' # 因为Namibia对应的国家代码为NA，这里需要显式声明一下，否则会被识别为nan
cancerdeath=pd.read_csv("./cancer-deaths-by-type.csv",encoding='utf-8-sig') # 读取文件
cancerdeath2019data = cancerdeath[cancerdeath['Year'] == 2019]  # 选取2019年的癌症死亡数据
countryname = cancerdeath2019data['Entity'].tolist()    # countryname为cancer-deaths-by-type.csv文件中的所有国家名称

demog= pd.read_csv("./demographics.csv",encoding='utf-8-sig')   # 读取文件demographics.csv
demog.at[15069,'location_key'] = 'NA'   # 这里也是一样，将该国家的代码显式声明为NA，避免被识别为nan
coun = []   # coun为所有计算了癌症死亡率的国家
ratio = []  # ratio为癌症死亡率
for country in countryname:
    num = 0
    if country in name_dict.keys(): # 若该国家名称在name_dict的键中，则求出该国家的癌症死亡数，并将该国家添加到coun中
        for i in range(3,32):   # i为3:32是为了对所有癌症死亡数求和
            num += cancerdeath2019data[cancerdeath2019data['Entity'] == country].iloc[0].tolist()[i]    # num为该国家的2019年各种癌症死亡总数
        coun.append(country)    # 将该国家添加到coun中
        ratio.append((num/int(demog[demog['location_key'] == name_dict[country]]['population']))*100000)    # 计算该国家的癌症死亡率并添加到ratio中
data=list(zip(coun,ratio))  # data为元素为元组的列表，每个元组由国家名称和癌症死亡率组成

In [5]:
huanbingrenshu= pd.read_csv("./number-of-people-with-cancer.csv",encoding='utf-8-sig')
huanbingrenshu = huanbingrenshu[huanbingrenshu['Year'] == 2017]
huanbingrenshu.columns = ['Entity','Code','Year','Number of patients']
cancerdeath2017data = cancerdeath[cancerdeath['Year'] == 2017]
countryname = cancerdeath2019data['Entity'].tolist()
coun = []
ratio = []
for country in countryname:
    num = 0
    if country in name_dict.keys(): # 若该国家名称在name_dict的键中，则求出该国家的癌症死亡数，并将该国家添加到coun中
        for i in range(3,32):   # i为3:32是为了对所有癌症死亡数求和
            num += cancerdeath2017data[cancerdeath2017data['Entity'] == country].iloc[0].tolist()[i]    # num为该国家的2017年各种癌症死亡总数
        coun.append(country)    # 将该国家添加到coun中
        a = num/huanbingrenshu[huanbingrenshu['Entity'] == country].iloc[:,3].to_list()
        a = list(a)
        ratio.append(a)    # 计算该国家的癌症死亡率并添加到ratio中
data=list(zip(coun,ratio))  # data为元素为元组的列表，每个元组由国家名称和癌症死亡率组成

In [6]:
imap=(
    Map(
        init_opts=opts.InitOpts(bg_color='rgba(255,250,205,0.2)',
                               width='1400px',
                                height='1000px',
                                page_title='2017年世界癌症死亡数据',
                                theme=ThemeType.ROMA
                               )
    )
    .add("癌症死亡率",data,"world",zoom=1)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="2017年世界癌症死亡数据--地图绘制"),
        legend_opts=opts.LegendOpts(is_show=True,selected_mode = True),
        visualmap_opts=opts.VisualMapOpts(
            is_piecewise=True,
            pieces = [
                {"min":1,},
                {"min":0.5,"max":1},
                {"min":0.25,"max":0.5},
                {"min":0.2,"max":0.25},
                {"min":0.15,"max":0.2},
                {"min":0.1,"max":0.15},
                {"max":0.1}
            ])
    )
 
)
imap.render_notebook()

In [16]:
cancer_incidence = pd.read_csv('cancer-incidence.csv')
cancer_incidence = cancer_incidence[cancer_incidence['Year'] == 2017]
countryname = cancer_incidence['Entity'].tolist()
coun = []   # coun为所有国家
ratio = []  # ratio为癌症发生率
for country in countryname:
    num = 0
    if country in name_dict.keys(): # 若该国家名称在name_dict的键中，则求出该国家的癌症死亡数，并将该国家添加到coun中
        coun.append(country)    # 将该国家添加到coun中
        a = cancer_incidence[cancer_incidence['Entity'] == country]['Incidence - Neoplasms - Sex: Both - Age: Age-standardized (Rate)'].to_list()
        a = list(a)
        ratio.append(a)    # 将癌症发生率添加到ratio中
data=list(zip(coun,ratio))  # data为元素为元组的列表，每个元组由国家名称和癌症死亡率组成

In [15]:
imap=(
    Map(
        init_opts=opts.InitOpts(bg_color='rgba(255,250,205,0.2)',
                               width='1400px',
                                height='1000px',
                                page_title='2017年世界癌症发生数据',
                                theme=ThemeType.ROMA
                               )
    )
    .add("癌症发生率",data,"world",zoom=1)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="2017年世界癌症发生数据--地图绘制"),
        legend_opts=opts.LegendOpts(is_show=True,selected_mode = True),
        visualmap_opts=opts.VisualMapOpts(
            is_piecewise=True,
            pieces = [
                {"min":500,},
                {"min":400,"max":500},
                {"min":300,"max":400},
                {"min":200,"max":300},
                {"min":100,"max":200},
                {"max":100}
            ])
    )
 
)
imap.render_notebook()